In [2]:
import numpy as np
import scipy.spatial.distance
from imutils.object_detection import non_max_suppression
import scipy.signal
from skimage.segmentation import slic
from skimage.util import img_as_float
from scipy.optimize import minimize

import os
import scipy.io
import cv2

from skimage import io
import itertools

In [3]:
def mkdir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [4]:
File_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach8/Output_Image_Dataset/Full_System_Output'
HS_files = sorted(os.listdir(File_path))
print("HS_files", HS_files)
# num = 0;
save_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach8/Output_Image_Dataset/Final_Output'

HS_files ['0001.jpg', '0002.jpg', '0006.jpg', '0008.jpg', '0009.jpg', '0010.jpg', '0012.jpg', '0014.jpg', '0015.jpg', '0016.jpg', '0017.jpg', '0018.jpg', '0019.jpg', '0020.jpg', '0021.jpg', '0022.jpg', '0024.jpg', '0026.jpg', '0027.jpg', '0028.jpg', '0029.jpg', '0031.jpg', '0032.jpg', '0033.jpg', '0034.jpg', '0036.jpg', '0037.jpg', '0038.jpg', '0039.jpg', '0040.jpg', '0041.jpg', '0042.jpg', '0043.jpg', '0044.jpg', '0045.jpg', '0046.jpg', '0047.jpg', '0050.jpg', '0051.jpg', '0052.jpg', '0053.jpg', '0054.jpg', '0056.jpg', '0058.jpg', '0061.jpg', '0062.jpg', '0063.jpg', '0064.jpg', '0065.jpg', '0068.jpg', '0069.jpg', '0070.jpg', '0071.jpg', '0072.jpg', '0074.jpg', '0076.jpg', '0077.jpg', '0078.jpg', '0079.jpg', '0082.jpg']


In [5]:
# print('file path', File_path)
file_list = os.listdir(File_path)
for file in file_list:
    file_name = file.rsplit('.', 1)
    input_img = os.path.join(File_path, file)
    #  Load the saliency map
    saliency_map = cv2.imread(input_img, cv2.IMREAD_GRAYSCALE)

    # Apply threshold to obtain the binary image
    # ret, saliency_map = cv2.threshold(saliency_map, 127, 255, cv2.THRESH_BINARY)

    # Threshold the saliency map to obtain a binary mask
    threshold = 128  # adjust this threshold as needed
    mask = (saliency_map > threshold).astype(np.uint8)

    # Find the connected components in the binary mask
    components, labels = cv2.connectedComponents(mask)

    # Initialize a list to store the bounding boxes and scores of each component
    boxes = []
    scores = []

    # Loop over each component and compute its bounding box and score
    for i in range(1, np.amax(components) + 1):
        # Extract the binary mask for the current component
        component_mask = (labels == i).astype(np.uint8)

        # Compute the bounding box of the component
        bbox = cv2.boundingRect(component_mask)

        # Compute the score of the component as the maximum saliency value within the component
        if np.count_nonzero(component_mask) > 0:
            score = np.max(saliency_map[component_mask == 1])
        else:
            score = 0

        # Append the bounding box and score to the list
        boxes.append(bbox)
        scores.append(score)

    # Apply non-maximum suppression to the list of bounding boxes and scores
    overlap_thresh = 0.5  # adjust this threshold as needed
    scores = np.array(scores)
    boxes = np.array(boxes)
    selected_indices = []

    # Flatten the scores array before performing argsort
    order = scores.ravel().argsort()[::-1]
    areas = (boxes[:, 2] * boxes[:, 3])

    if np.any(areas == 0):
        areas[areas == 0] = 1

    # Loop over the indices in the order determined by the scores and compute the overlap of each bounding box with the previously selected bounding boxes
    while order.size > 0:
        # Select the bounding box with the highest score and add it to the list of selected boxes
        i = order[0]
        selected_indices.append(i)

        # Compute the overlap of the selected bounding box with the other bounding boxes
        xx1 = np.maximum(boxes[i, 0], boxes[order[1:], 0])
        yy1 = np.maximum(boxes[i, 1], boxes[order[1:], 1])
        xx2 = np.minimum(boxes[i, 0] + boxes[i, 2], boxes[order[1:], 0] + boxes[order[1:], 2])
        yy2 = np.minimum(boxes[i, 1] + boxes[i, 3], boxes[order[1:], 1] + boxes[order[1:], 3])
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[order[1:]]

        # Remove bounding boxes with overlap greater than the overlap threshold
        inds = np.where(overlap <= overlap_thresh)[0]
        order = order[inds + 1]

    # Loop over the selected bounding boxes and draw them on the image
    img_rgb = cv2.cvtColor(saliency_map, cv2.COLOR_GRAY2RGB)
    for i in selected_indices:
        x, y, w, h = boxes[i]
        if w > 0 and h > 0:
            area = img_rgb[y:y+h, x:x+w]
            gray_roi = np.zeros((h, w), dtype=np.uint8)
            white_roi = 255 - gray_roi
            img_rgb[y:y+h, x:x+w] = cv2.cvtColor(white_roi, cv2.COLOR_GRAY2BGR)
        
        # cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)

    save_im_final_featureMap = save_path + '/' + file_name[0] + '.png'
    cv2.imwrite(save_im_final_featureMap, img_rgb)